In [1]:
#LightGBM
#应先安装lightgbm，pip install lightgbm即可
#但官网上给的安装信息有点太复杂了，详见https://lightgbm.readthedocs.io/en/latest/Installation-Guide.html

#下面这个网页介绍了使用lightgbm的两种形式：原生形式(import lightgbm as lgb)
#和sklearn接口形式(from lightgbm import LGBMRegressor, LGBMClassifier)
#具体可查看https://www.cnblogs.com/chenxiangzhen/p/10894306.html
#原生形式中可以使用lgb.cv做交叉验证选参数， 但要注意数据集必须使用lgb.Dataset函数加以转换
#为了在我们熟悉的sklearn框架下使用，下面仅介绍sklearn接口形式

#lightgbm参数很多，应仔细阅读https://lightgbm.readthedocs.io/en/latest/Parameters.html
#关于调参，可以参考https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html
#基于上述网页，我整理了一些重要参数，分为几类：
#1、核心参数：task, objective, boosting, n_estimators, learning_rate, metric
#2、与决策树相关的参数：num_leaves, max_depth, min_data_in_leaf, feature_fraction_bynode, min_gain_split
#3、涉及加速与防止过拟合的参数：bagging_fraction, feature_fraction, lambda_l1, lambda_l2, max_bin, 
#   min_data_in_bin, bin_construct_sample_cnt
#（实际上，决策树中的参数max_depth, min_data_in_leaf, feature_fraction_bynode也有防止过拟合的作用）
#4、处理不平衡的参数：pos_bagging_fraction, neg_bagging_fraction, is_unbalance
#5、GOSS相关参数（设置boosting=goss才会启用GOSS）：top_rate, other_rate
#6、EFB相关参数：enable_bundle, max_conflict_rate （实际上，这两个参数也可以实现加速）


#网上也有很多调参攻略，例如我随便搜索看到的网页：
#https://www.cnblogs.com/wzdLY/p/9867719.html
#https://blog.csdn.net/u012513618/article/details/78441676
#https://www.cnblogs.com/jiangxinyang/p/9337094.html
#https://www.imooc.com/article/43784?block_id=tuijian_wz

#不需要处理缺失值；不需要独热编码（但不能输入字符串）

In [3]:
#pip install lightgbm

Note: you may need to restart the kernel to use updated packages.


In [3]:
#pip install upgrade pandas -user -i https://pypi.tuna.tsinghua.edu.cn/simple

Note: you may need to restart the kernel to use updated packages.



Usage:   
  c:\ProgramData\Anaconda3\python.exe -m pip install [options] <requirement specifier> [package-index-options] ...
  c:\ProgramData\Anaconda3\python.exe -m pip install [options] -r <requirements file> [package-index-options] ...
  c:\ProgramData\Anaconda3\python.exe -m pip install [options] [-e] <vcs project url> ...
  c:\ProgramData\Anaconda3\python.exe -m pip install [options] [-e] <local project path> ...
  c:\ProgramData\Anaconda3\python.exe -m pip install [options] <archive url/path> ...

no such option: -u


In [2]:
import pandas
pandas.__version__
pandas.Int64Index

'2.0.1'

In [3]:
import pandas as pd
import numpy as np
import os
from lightgbm import LGBMRegressor,LGBMClassifier
from sklearn.metrics import r2_score, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

# 使用computer数据的例子，分类问题
alldata_X = pd.read_csv(r'C:\Users\jessica\Desktop\统计机器学习\Bootstrap1_feature_X.csv')
alldata_Y = pd.read_csv(r'C:\Users\jessica\Desktop\统计机器学习\Bootstrap1_feature_Y.csv')

X_train, X_test, y_train, y_test = train_test_split(
    alldata_X, alldata_Y, test_size=1/3, random_state=1) 

AttributeError: module 'pandas' has no attribute 'Int64Index'

In [2]:
#尝试发现对于DataFrame，必须将自变量中的定性变量设置为‘category’，才能在LightGBM中自动识别字符串。
#但对于y，LightGBM会自动对y重编码，因而LightGBM可接受字符串形式的y，且不管它是何类型。
#因此，下面转换X中的定性变量为‘category’。
category = list(X_train.select_dtypes(include=['object']).columns)
X_train[category] = X_train[category].astype('category')
X_train.dtypes

geo_index                   float64
floor_height                float64
population_dense            float64
count_floors_pre_eq           int64
age                           int64
                             ...   
plan_configuration_u          int64
legal_ownership_status_a      int64
legal_ownership_status_r      int64
legal_ownership_status_v      int64
legal_ownership_status_w      int64
Length: 68, dtype: object

In [3]:
clf = LGBMClassifier(objective="multiclass",min_child_samples=1,random_state=1,metric="multiclass")
clf.fit(X_train, y_train, categorical_feature=category) 
#LightGBM会将categorical_feature中的变量作为定性变量处理
#对于DataFrame，LightGBM只会自动识别类型为'category’的变量为定性变量。
#因此，这里可以不必设置categorical_feature（如果不设置，下面的报警也就不会出现），运行结果也一致
#但建议设置categorical_feature。如果未将DataFrame中的数值型定性变量转换为'category’，
#并且未在categorical_feature中声明，那么LightGBM会作为定量变量处理，这就会产生错误的结果，而且也没有任何报警提示

clf.predict_proba(X_train)

c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')


array([[0.83159659, 0.12949816, 0.03890526],
       [0.91416878, 0.08277065, 0.00306057],
       [0.70548703, 0.26868246, 0.02583051],
       ...,
       [0.01879401, 0.2762871 , 0.70491889],
       [0.04289312, 0.57032594, 0.38678094],
       [0.702374  , 0.24304378, 0.05458222]])

In [4]:
y_pred=clf.predict(X_test)
sum(y_pred==y_test.values.reshape(-1))/len(y_pred)

0.7075793037859421

In [5]:
# 网格搜索，参数优化
#这里需要注意，GridSearchCV()中无法设置LightGBM模型fit中的categorical_feature，必须将定性变量设置为'category'！

estimator = LGBMClassifier(objective='multiclass',min_child_samples=1,random_state=1,metric="multiclass") 
#因为这个例子中的数据很少，所以用了min_child_samples=1
param_grid = {
    'learning_rate': [0.01, 0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1],
    'n_estimators': [20, 40],
    'num_leaves':[4,6,8,10,12]
}
gs = GridSearchCV(estimator, param_grid,cv=StratifiedKFold(n_splits=2, random_state=1, shuffle=True))
gs.fit(X_train, y_train)
print('Best parameters found by grid search are:', gs.best_params_)

c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packag

Best parameters found by grid search are: {'learning_rate': 0.8, 'n_estimators': 40, 'num_leaves': 12}


In [6]:
gs.best_score_#这是auc

0.6428571428571428

In [6]:
gs.best_estimator_

LGBMClassifier(learning_rate=0.8, metric='multiclass', min_child_samples=1,
               n_estimators=40, num_leaves=12, objective='multiclass',
               random_state=1)

In [8]:
# 使用最优参数建模
gbm = gs.best_estimator_
gbm.fit(X_train, y_train)
# 看拟合    效果
y_pred=gbm.predict(X_test) #这个例子数据少，所以就用所有数据去回测，实际上应该使用测试集
y_pred

c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\jessica\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


array([3, 2, 3, ..., 1, 3, 3], dtype=int64)

In [9]:
len(y_pred)


148259

In [11]:
confusion_matrix(y_true=y_test,y_pred=y_pred)
sum(y_pred==y_test.values.reshape(-1))/len(y_pred)



0.7022440458926608

In [10]:
f1_score(y_true=y_train,y_pred=gbm.predict(X_train.iloc[0:14,]),pos_label='yes')

0.782608695652174

In [11]:
# 变量相对重要性
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [26, 0, 20, 14]


In [12]:
#查看随着基学习器个数的增加，集成模型效果的变化
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train,y_train, test_size=1/3, random_state=1) #划分训练集和验证集

gbm = gs.best_estimator_
gbm.fit(X_train2, y_train2,  eval_metric={ 'auc'}, 
        early_stopping_rounds=5, eval_set=[(X_val, y_val)]) #虽然设置了以auc为评价指标，但它还是用了binary_logloss
#可见，由于binary_logloss变大，它判断为出现了过拟合，因而在设定的early_stopping_rounds次过拟合后就停止了
#“best iteration is:[1]”表示n_estimators为1是最好的，但它只是一种展示，并没有改变之前的gbm参数n_estimators

[1]	valid_0's binary_logloss: 0.634353
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.636342
[3]	valid_0's binary_logloss: 0.63838
[4]	valid_0's binary_logloss: 0.640466
[5]	valid_0's binary_logloss: 0.642598
[6]	valid_0's binary_logloss: 0.644774
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.634353


C:\Users\liyan\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\liyan\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


LGBMClassifier(learning_rate=0.01, min_child_samples=1, n_estimators=20,
               num_leaves=4, objective='binary', random_state=1)

In [13]:
# 若用所有的原始数据建模，并基于这些数据验证，将会完全拟合正确。此时，由于评价指标值一直变好，所以不会提前终止。
gbm = gs.best_estimator_
gbm.fit(X_train, y_train, eval_metric={ 'auc','binary_logloss'}, 
        early_stopping_rounds=5,eval_set=[(X_train, y_train)]) 
#结果是没有达到early stopping的条件。

[1]	training's binary_logloss: 0.645831
Training until validation scores don't improve for 5 rounds
[2]	training's binary_logloss: 0.640011
[3]	training's binary_logloss: 0.63431
[4]	training's binary_logloss: 0.628725
[5]	training's binary_logloss: 0.623251
[6]	training's binary_logloss: 0.617881
[7]	training's binary_logloss: 0.612607
[8]	training's binary_logloss: 0.607436
[9]	training's binary_logloss: 0.602366
[10]	training's binary_logloss: 0.597387
[11]	training's binary_logloss: 0.592495
[12]	training's binary_logloss: 0.587695
[13]	training's binary_logloss: 0.582986
[14]	training's binary_logloss: 0.578351
[15]	training's binary_logloss: 0.573803
[16]	training's binary_logloss: 0.569338
[17]	training's binary_logloss: 0.564948
[18]	training's binary_logloss: 0.560632
[19]	training's binary_logloss: 0.556393
[20]	training's binary_logloss: 0.552226
Did not meet early stopping. Best iteration is:
[20]	training's binary_logloss: 0.552226


LGBMClassifier(learning_rate=0.01, min_child_samples=1, n_estimators=20,
               num_leaves=4, objective='binary', random_state=1)

In [14]:
# 使用car.test.frame数据的例子，回归问题
os.chdir(r"C:\Users\liyan\Documents\教学\数据挖掘\2019秋季\python\data") #切换路径
alldata = pd.read_csv('car.csv')

y = alldata[['Mileage']] #将Mileage作为因变量y
X = alldata[['Country', 'Reliability','Price' , 'Type',
       'Weight', 'Disp.', 'HP']] #取出7个自变量，存储为X

#下面设置X中的定性变量为'category'
category = list(X.select_dtypes(include=['object']).columns)
X[category] = X[category].astype('category')

#划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X,y, test_size=1/3, random_state=1) 
X_train.dtypes

C:\Users\liyan\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Country        category
Reliability     float64
Price             int64
Type           category
Weight            int64
Disp.             int64
HP                int64
dtype: object

In [15]:
# 网格搜索，参数优化
#再次提醒，GridSearchCV()中无法设置LightGBM模型fit中的categorical_feature，必须将定性变量设置为'category'！

estimator = LGBMRegressor(min_child_samples=5,random_state=1)
param_grid = {
    'learning_rate': [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8],
    'n_estimators': [20, 40, 60],
    'num_leaves':[2,3,4,5,10],
}
gs = GridSearchCV(estimator, param_grid,cv=3)
gs.fit(X_train, y_train)
print('Best parameters found by grid search are:', gs.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.3, 'n_estimators': 20, 'num_leaves': 5}


In [16]:
# 用最优参数建模
gbm = gs.best_estimator_.fit(X_train, y_train)

# 变量重要性
print('Feature importances:', list(gbm.feature_importances_))

Feature importances: [8, 17, 22, 0, 5, 15, 13]


In [17]:
y_pred =  gs.best_estimator_.predict(X_test)
r2_score(y_test,y_pred)

0.7405865222135871

In [18]:
#查看随着基学习器个数的增加，集成模型效果的变化
#划分训练集和验证集
X_train2, X_val, y_train2, y_val = train_test_split(
    X_train,y_train, test_size=1/3, random_state=1) 
gbm = gs.best_estimator_
gbm.fit(X_train2, y_train2, early_stopping_rounds=5, eval_set=[(X_val, y_val)]) 
#这表明n_estimators还可以减小，后续调参略

[1]	valid_0's l2: 18.1726
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l2: 14.606
[3]	valid_0's l2: 11.3205
[4]	valid_0's l2: 9.7778
[5]	valid_0's l2: 8.98449
[6]	valid_0's l2: 8.98612
[7]	valid_0's l2: 9.63456
[8]	valid_0's l2: 9.72916
[9]	valid_0's l2: 10.0802
[10]	valid_0's l2: 9.48295
Early stopping, best iteration is:
[5]	valid_0's l2: 8.98449


C:\Users\liyan\anaconda3\lib\site-packages\lightgbm\basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\liyan\anaconda3\lib\site-packages\lightgbm\basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


LGBMRegressor(learning_rate=0.3, min_child_samples=5, n_estimators=20,
              num_leaves=5, random_state=1)

#### 关于LightGBM是否可放入Pipeline的问题：

对于含有定性自变量的数据：我们无法在Pipeline的fit中设置categorical_feature；即便我们在预处理中将字符串编码为数值，但无法在preprocessor中设置定性变量为“category”。这会导致对定性变量处理的错误，使得我们无法正确地将LightGBM放入Pipeline中。

对于不含有定性自变量的数据：尝试发现，LightGBM可以正常放入Pipeline中。

In [38]:
import pandas as pd
import numpy as np

In [39]:
df = pd.read_csv('train_values.csv')

In [44]:
df1 = df.iloc[:,15:].astype(int)

In [46]:
notz = []
for i in range(len(df1.columns)):
    notz.append(sum(df1.iloc[:,i]!=0)/len(df1.iloc[:,1]))
notz

[0.08864509345704737,
 0.7619349119918957,
 0.03433217831090441,
 0.0182347726984931,
 0.06815399787414476,
 0.07526832207090534,
 0.2549875096411756,
 0.08501118568232663,
 0.042590013085137816,
 0.015859494015755888,
 0.014984593305474653,
 0.9199465850092671,
 0.11187984696912137,
 0.06437811059819418,
 0.03362611808857219,
 0.008100506137735466,
 0.0009401345351706249,
 0.0003607046788001581,
 0.0010706021849494053,
 0.00018802690703412496,
 0.00014581678504687243,
 8.825752779152804e-05,
 0.005118936611908626]

In [42]:
conflict = []
for i in range(1,len(df1.columns)):
    conflict.append(sum(df1.iloc[:,-3]!=df1.iloc[:,i])/len(df1.iloc[:,1]))
conflict

[0.7619809593976999,
 0.03445497139304914,
 0.018334542077735695,
 0.06829981465919163,
 0.0753450677472458,
 0.2551026281556863,
 0.08514932789973945,
 0.04269745703201446,
 0.015905541421560164,
 0.015130410090521525,
 0.9199312358739989,
 0.1117340301840745,
 0.06452392738324104,
 0.033771934873619056,
 0.008246322922782337,
 0.0010859513202174972,
 0.0005065214638470306,
 0.0012164189699962779,
 0.0003338436920809974,
 0.0,
 0.00023407431283840047,
 0.005257078829321453]

In [32]:
df1.iloc[:,-3] = df1.iloc[:,-3] + df1.iloc[:,0]

In [34]:
notz = []
for i in range(len(df1.columns)):
    notz.append(sum(df1.iloc[:,i]!=0)/len(df1.iloc[:,1]))
notz

[0.08864509345704737,
 0.7619349119918957,
 0.03433217831090441,
 0.0182347726984931,
 0.06815399787414476,
 0.07526832207090534,
 0.2549875096411756,
 0.08501118568232663,
 0.042590013085137816,
 0.015859494015755888,
 0.014984593305474653,
 1.0,
 0.9199465850092671,
 0.11187984696912137,
 0.06437811059819418,
 0.03362611808857219,
 0.008100506137735466,
 0.0009401345351706249,
 0.0003607046788001581,
 0.0010706021849494053,
 0.00018802690703412496,
 0.015130410090521525,
 8.825752779152804e-05,
 0.005118936611908626]

In [36]:
conflict = []
for i in range(1,len(df1.columns)):
    conflict.append(sum(df1.iloc[:,-4]!=df1.iloc[:,i])/len(df1.iloc[:,1]))
conflict

[0.7619387492757127,
 0.034520205217938534,
 0.018399775902625086,
 0.0683113265106427,
 0.0753642541663309,
 0.25513716371003947,
 0.08519921258936075,
 0.04271664345109957,
 0.016001473516985738,
 0.015164945644874733,
 1.0,
 0.9198890257520117,
 0.11169182006208725,
 0.0645661375052283,
 0.03381414499560631,
 0.00828853304476959,
 0.0011281614422047497,
 0.0005487315858342831,
 0.0012586290919835304,
 0.0,
 0.015310762429921604,
 0.00027628443482565304,
 0.005306963518942751]

In [ ]:
df1.iloc[:,-4] = df1.iloc[:,-4] + df1.iloc[:,10]

In [50]:
import pandas as pd

# 计算每列中0的比例
zero_ratio = (df1 == 0).mean()

# 按照0的比例降序排序
sorted_columns = zero_ratio.sort_values(ascending=False).index

# 根据排序结果创建新的DataFrame
df2 = df1[sorted_columns]

# 打印新的DataFrame
df2


,has_secondary_use_use_police,has_secondary_use_gov_office,has_secondary_use_health_post,has_secondary_use_school,has_secondary_use_institution,has_secondary_use_industry,has_secondary_use_other,has_secondary_use_rental,has_superstructure_other,has_superstructure_rc_engineered,...,has_superstructure_rc_non_engineered,has_secondary_use_agriculture,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_bamboo,has_superstructure_adobe_mud,has_secondary_use,has_superstructure_timber,has_superstructure_mud_mortar_stone,count_families
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [84]:
features = df2.columns.tolist()
# df2[features]
# features
# features[0]
# (df2[features[0]] == 0 and df2[features[1:]] == 0)
# df2[features[1:]]

pd.concat([df2.loc[:,[features[0],features[i]]].apply(lambda x: sum(x)==0,axis=1) for i in range(1,len(features))],axis=1)

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,False,False
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,False,False,False
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
260597,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
260598,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,False,False
260599,True,True,True,True,True,True,True,True,True,True,...,True,True,True,False,True,True,True,True,True,False


In [88]:
zero_ratio =pd.concat([df2.loc[:,[features[0],features[i]]].apply(lambda x: sum(x)==0,axis=1) for i in range(1,len(features))],axis=1)

In [90]:
import pandas as pd
import numpy as np

# 贪心算法进行特征绑定（EFB）
def greedy_efb(df):
    features = df.columns.tolist()  # 获取所有特征列表
    binding_features = {}  # 存储绑定特征的字典

    while len(features) > 0:
        current_feature = features[0]  # 当前待绑定的特征
        bound_features = [current_feature]  # 存储与当前特征绑定的特征列表

        # 计算当前特征与其他特征同时为0的比例
        zero_ratio = pd.concat([df2.loc[:,[features[0],features[i]]].apply(lambda x: sum(x)==0,axis=1) for i in range(1,len(features))],axis=1)

        zero_ratio = zero_ratio.mean()

        # 找到同时为0比例最高的特征
        max_zero_ratio_feature = zero_ratio.idxmax()
        max_zero_ratio = zero_ratio[max_zero_ratio_feature]

        # 将同时为0比例最高的特征加入绑定特征列表
        if max_zero_ratio > 0:
            bound_features.append(max_zero_ratio_feature)

        # 从特征列表中移除已绑定的特征
        features = [f for f in features if f not in bound_features]

        # 存储当前特征及其绑定的特征列表
        binding_features[current_feature] = bound_features

    return binding_features

# 使用贪心算法进行特征绑定（EFB）
binding_features = greedy_efb(df2)

# 打印绑定特征及其绑定的特征集合
for feature, bound_features in binding_features.items():
    print("Binding Feature:", feature)
    print("Bound Features:", bound_features)
    print()


ValueError: No objects to concatenate

In [58]:
df2

,has_secondary_use_use_police,has_secondary_use_gov_office,has_secondary_use_health_post,has_secondary_use_school,has_secondary_use_institution,has_secondary_use_industry,has_secondary_use_other,has_secondary_use_rental,has_superstructure_other,has_superstructure_rc_engineered,...,has_superstructure_rc_non_engineered,has_secondary_use_agriculture,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_bamboo,has_superstructure_adobe_mud,has_secondary_use,has_superstructure_timber,has_superstructure_mud_mortar_stone,count_families
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,1,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260596,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260597,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260598,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
260599,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,1


In [55]:
len(df2.index)

260601

In [57]:
import pandas as pd
import numpy as np
df3 = df2.iloc[:2000,:]
def greedy_efb(df, threshold):
    features = df.columns.tolist()  
    binding_features = {} 

    while len(features) > 0:
        current_feature = features[0]  
        bound_features = [current_feature] 

        zero_ratio = (df[current_feature] == 0) & (df[features[1:]] == 0)
        zero_ratio = zero_ratio.mean()
        max_zero_ratio_feature = zero_ratio.idxmax()
        max_zero_ratio = zero_ratio[max_zero_ratio_feature]


        if max_zero_ratio >= threshold:
            bound_features.append(max_zero_ratio_feature)

        features = [f for f in features if f not in bound_features]

        binding_features[current_feature] = bound_features

    return binding_features

threshold = 0.5

binding_features = greedy_efb(df3, threshold)

for feature, bound_features in binding_features.items():
    print("Binding Feature:", feature)
    print("Bound Features:", bound_features)
    print()


Binding Feature: has_secondary_use_use_police
Bound Features: ['has_secondary_use_use_police']

Binding Feature: has_secondary_use_gov_office
Bound Features: ['has_secondary_use_gov_office']

Binding Feature: has_secondary_use_health_post
Bound Features: ['has_secondary_use_health_post']

Binding Feature: has_secondary_use_school
Bound Features: ['has_secondary_use_school']

Binding Feature: has_secondary_use_institution
Bound Features: ['has_secondary_use_institution']

Binding Feature: has_secondary_use_industry
Bound Features: ['has_secondary_use_industry']

Binding Feature: has_secondary_use_other
Bound Features: ['has_secondary_use_other']

Binding Feature: has_secondary_use_rental
Bound Features: ['has_secondary_use_rental']

Binding Feature: has_superstructure_other
Bound Features: ['has_superstructure_other']

Binding Feature: has_superstructure_rc_engineered
Bound Features: ['has_superstructure_rc_engineered']

Binding Feature: has_superstructure_cement_mortar_stone
Bound Feat

In [89]:
import pandas as pd

# 示例数据
data = {'A': [0, 1, 0, 1, 0],
        'B': [0, 0, 1, 1, 0],
        'C': [1, 0, 0, 1, 1],
        'D': [0, 1, 1, 0, 1],
        'E': [0, 1, 1, 1, 0]}
df = pd.DataFrame(data)

# 贪心算法进行特征绑定（EFB）
def greedy_efb(df):
    features = df.columns.tolist()  # 获取所有特征列表
    binding_features = {}  # 存储绑定特征的字典

    while len(features) > 0:
        current_feature = features[0]  # 当前待绑定的特征
        bound_features = [current_feature]  # 存储与当前特征绑定的特征列表

        for feature in features[1:]:
            # 计算当前特征与其他特征同时为0的比例
            zero_ratio = (df[current_feature] == 0) & (df[feature] == 0)
            zero_ratio = zero_ratio.mean()

            # 设置阈值
            threshold = 0.5

            # 将同时为0比例超过阈值的特征加入绑定特征列表
            if zero_ratio > threshold:
                bound_features.append(feature)

        # 从特征列表中移除已绑定的特征
        features = [f for f in features if f not in bound_features]

        # 存储当前特征及其绑定的特征列表
        binding_features[current_feature] = bound_features

    return binding_features

# 使用贪心算法进行特征绑定（EFB）
binding_features = greedy_efb(df3)

# 打印绑定特征及其绑定的特征集合
for feature, bound_features in binding_features.items():
    print("Binding Feature:", feature)
    print("Bound Features:", bound_features)
    print()


Binding Feature: has_secondary_use_use_police
Bound Features: ['has_secondary_use_use_police', 'has_secondary_use_gov_office', 'has_secondary_use_health_post', 'has_secondary_use_school', 'has_secondary_use_institution', 'has_secondary_use_industry', 'has_secondary_use_other', 'has_secondary_use_rental', 'has_superstructure_other', 'has_superstructure_rc_engineered', 'has_superstructure_cement_mortar_stone', 'has_secondary_use_hotel', 'has_superstructure_stone_flag', 'has_superstructure_rc_non_engineered', 'has_secondary_use_agriculture', 'has_superstructure_mud_mortar_brick', 'has_superstructure_cement_mortar_brick', 'has_superstructure_bamboo', 'has_superstructure_adobe_mud', 'has_secondary_use', 'has_superstructure_timber']

Binding Feature: has_superstructure_mud_mortar_stone
Bound Features: ['has_superstructure_mud_mortar_stone']

Binding Feature: count_families
Bound Features: ['count_families']

